In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import geojson

ModuleNotFoundError: No module named 'geojson'

In [2]:
# Access built-in Natural Earth data via GeoPandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

ERROR 1: PROJ: proj_create_from_database: Open of /home/ankur/miniconda3/share/proj failed


In [3]:
# Get a list (dataframe) of country centroids
centroids = world.centroid
centroids = pd.Series([centroid.coords[0][1] for centroid in centroids])
centroid_df = pd.concat([world.name, centroids, world.iso_a3], axis=1)
centroid_df.rename(columns={0: "centroid_latitude"}, inplace=True)

/tmp/ipykernel_2346/3765168726.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = world.centroid


In [4]:
centroid_df.to_csv("./data/latitude centroid/country_centroid_latitude.csv", index=False)

In [5]:
spi_data = pd.read_csv("./data/SPI/SPI_index_mean.csv")

In [6]:
set1 = set(centroid_df.iso_a3)
filtered_centroid_data = []
for i, row in spi_data.iterrows():
    country_code = row["iso3c"]
    if country_code in set1:
        centroid = centroid_df.loc[centroid_df["iso_a3"] == country_code, 'centroid_latitude'].iloc[0]
    else:
        centroid = np.nan
    filtered_centroid_data.append(centroid)
    
spi_data["centroid_latitude"] = filtered_centroid_data

In [7]:
countries_left = set(spi_data.iso3c) - set(centroid_df.iso_a3)

In [8]:
country_codes_df = pd.read_csv("./data/country_codes2.csv")
country_codes_df = country_codes_df.apply(lambda x: x.str.replace('"', ""))
country_codes_df = country_codes_df.apply(lambda x: x.str.replace(' ', ""))
country_codes_df.head(5)

,Country,Alpha-2 code,Alpha-3 code,Numeric code,Latitude (average),Longitude (average)
0,Afghanistan,AF,AFG,4,33,65
1,Albania,AL,ALB,8,41,20
2,Algeria,DZ,DZA,12,28,3
3,AmericanSamoa,AS,ASM,16,-14.3333,-170
4,Andorra,AD,AND,20,42.5,1.6


In [9]:
# https://www.npmjs.com/package/world-countries-centroids
centroid_df_github = pd.read_csv("./data/latitude centroid/country_centroid_latitude_from_github.csv")
centroid_df_github.head(3)

,longitude,latitude,COUNTRY,ISO,COUNTRYAFF,AFF_ISO
0,-170.700732,-14.305712,American Samoa,AS,United States,US
1,166.638003,19.302046,United States Minor Outlying Islands,UM,United States,US
2,-159.787689,-21.222613,Cook Islands,CK,New Zealand,NZ


In [10]:
# tmp = 
for country_code in countries_left:
    alpha2_code = country_codes_df.loc[country_codes_df["Alpha-3 code"] == country_code, "Alpha-2 code"]
    if len(alpha2_code) != 0:
        alpha2_code = alpha2_code.iloc[0]
        centroid = centroid_df_github[centroid_df_github["AFF_ISO"] == alpha2_code]["latitude"]
        
        if len(centroid) != 0:
            centroid = centroid.iloc[0]
            spi_data.loc[spi_data["iso3c"]==country_code, 'centroid_latitude'] = centroid

In [ ]:
spi_data.dropna(subset=["centroid_latitude", "spi_mean"], inplace=True)

In [13]:
tropic_of_cancer = 23.43626
tropic_of_capricorn = -1.0 * tropic_of_cancer

(23.43626, -23.43626)

In [15]:
spi_data.head()

,country,spi_mean,spi_std,income,iso3c,spi_category,centroid_latitude
0,Canada,86.69,0.53,High income,CAN,1,61.469076
1,Sao Tome and Principe,46.29,4.57,Lower middle income,STP,3,0.227447
2,Turkmenistan,21.63,1.75,Upper middle income,TKM,3,39.091240
3,Lao PDR,49.00,3.89,Lower middle income,LAO,3,18.444978
4,Lithuania,81.90,3.80,High income,LTU,1,55.284319


## Categorising countries based on their centroid's latitude

### Categories:
#### 1. If centroid lies within tropic of cancer and tropic of capricorn 
#### 2. otherwise

In [18]:
spi_data["centroid_category"] = spi_data.apply(lambda row: 1 if (row.centroid_latitude <= tropic_of_cancer and 
                                 row.centroid_latitude >= tropic_of_capricorn) else 2, axis=1)

In [23]:
spi_data[spi_data["country"] == "India"]

,country,spi_mean,spi_std,income,iso3c,spi_category,centroid_latitude,centroid_category
48,India,67.87,2.46,Lower middle income,IND,2,22.925006,1


In [24]:
spi_data.to_csv("./data/SPI_index_mean_and_centroid_latitude.csv", index=False)